In [ ]:
import requests, os, json
ALCHEMY_KEY = os.getenv('ALCHEMY_KEY')
MEETUP_KEY = os.getenv('MEETUP_KEY')
TOPIC = os.getenv('TOPIC', 'bluemix')

In [ ]:
# GET /locations
'''
Check an arbitrary string against the Alchemy entity extraction, using the set key
'''
resp = requests.post('http://gateway-a.watsonplatform.net/calls/text/TextGetRankedNamedEntities', params=dict(
        apikey=ALCHEMY_KEY, 
        text=json.loads(REQUEST)['args']['where'],
        outputMode='json'
))
resp.raise_for_status()
named_entities = resp.json()['entities']
resp.text

In [ ]:
# GET /groups
# find active groups using the location search to get more accurate location info
resp = requests.get('https://api.meetup.com/find/groups', params=dict(
        location=json.loads(REQUEST)['args']['city'],
        text=TOPIC,
        order='distance',
        key=MEETUP_KEY,
        sign='true'
    ))
resp.raise_for_status()
resp.json()
group_locations = [
    {'city':group['city'], 'state':group['state'], 'country':group['country']} for group in resp.json()
]
json.dumps(group_locations)

In [ ]:
# GET /echo
REQUEST

In [ ]:
# POST /echo
REQUEST

In [ ]:
# POST /generate_search_url
'''
Find city information from the posted body, use that the find
nearby open groups, and use the group location to do a better
search for upcoming open events.
'''
# find the city's name
resp = requests.get('http://gateway-a.watsonplatform.net/calls/text/TextGetRankedNamedEntities', params=dict(
        apikey=ALCHEMY_KEY, 
        text=json.loads(REQUEST)['body']['text'],
        outputMode='json'
))
resp.raise_for_status()
named_entities = resp.json()['entities']
cities = [
    entity['text'] for entity in named_entities
        if entity['type'] == 'City'
]

mentioned_city = cities[0]

# find nearby active open groups
resp = requests.get('https://api.meetup.com/find/groups', params=dict(
        location=mentioned_city,
        text=TOPIC,
        order='distance',
        key=MEETUP_KEY,
        sign='true'
    ))
resp.raise_for_status()

# use the closest group's location info for the event search 
group_locations = [
    {'city':group['city'], 'state':group['state'], 'country':group['country']} for group in resp.json()
]

result = '{}'
if len(group_locations) > 0:
    resp2 = requests.get('http://api.meetup.com/2/open_events', params=dict(
        key=MEETUP_KEY,
        city=group_locations[0]['city'],
        state=group_locations[0]['state'],
        country=group_locations[0]['country'],
        text=TOPIC,
        sign='true',
        radius=50
    ))
    resp2.raise_for_status()
    result = resp2.json()['meta']['signed_url']

result